In [7]:
import geopandas as gpd
import pandas as pd
import json 
import psycopg2 

In [17]:
#get shark attacks geojson from inputted coordinates
   #query that gives a Feature Collection object
query_shark_attacks = '''SELECT jsonb_build_object(
      'type',     'FeatureCollection',
      'features', jsonb_agg(features.feature)
   )
   FROM (
   SELECT jsonb_build_object(
      'type',       'Feature',
      'id',          id,
      'geometry',   ST_AsGeoJSON(geometry)::jsonb,
      'properties', to_jsonb(inputs) - 'id' - 'geometry'
   ) AS feature
   FROM (SELECT 
      s."Date", s."Time", s."Location", s."Location_attack", s."Location_attack2", s.full_location, s."Location_attack3", s."Activity", s."Shark", s.geometry, s.id
      FROM shark_attacks as s
      WHERE st_intersects(s.geometry, 
                  st_transform(
                     st_buffer(
                        st_transform(
                           st_setsrid(
                              st_makepoint(-158.073760, 21.616998),4326),26904),20000),4326)
                  )
   ) inputs) features'''

In [19]:
connection = psycopg2.connect(database="geotech_ocean_haz", user="postgres", password = "postgres")

In [20]:
 #Reads the query and store it in a dataframe
feature_collection_shark_attacks = pd.read_sql(query_shark_attacks, connection)

C:\Users\johnk\anaconda3\envs\ocean_haz\lib\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [21]:
   #Getting geojson dictionary by calling iloc[0] on the jsonb_build_object column
   feature_collection_dict_shark_attacks = feature_collection_shark_attacks.iloc[0]['jsonb_build_object']

In [22]:
 #Converting to geojson
shark_feature_collection = json.dumps(feature_collection_dict_shark_attacks)
print(shark_feature_collection)

{"type": "FeatureCollection", "features": [{"id": 4, "type": "Feature", "geometry": {"type": "Point", "coordinates": [-158.036483727, 21.48343645]}, "properties": {"Date": "03/05/2021", "Time": " 3:30 pm", "Shark": "Requiem shark, length 5-6 feet", "Activity": "Surfing", "Location": "Oahu", "full_location": "Oahu, Kaneohe", "Location_attack": " Kaneohe", "Location_attack2": " North Beach", "Location_attack3": " approx 10 yds from shore"}}, {"id": 17, "type": "Feature", "geometry": {"type": "Point", "coordinates": [-158.036483727, 21.48343645]}, "properties": {"Date": "18/09/2019", "Time": " 10:10 am", "Shark": "Galapagos shark, length 5-9 feet.", "Activity": "Swimming with sharks", "Location": "Oahu", "full_location": "Oahu, Haleiwa", "Location_attack": " Haleiwa", "Location_attack2": " approx 3 miles from shore", "Location_attack3": null}}, {"id": 18, "type": "Feature", "geometry": {"type": "Point", "coordinates": [-158.036483727, 21.48343645]}, "properties": {"Date": "18/09/2019", "T